In [1]:
import os, glob, math, json, random
import numpy as np
from tqdm import tqdm

In [2]:
def load_segments_with_subject_ids(root_dir, eeg_suffix='EEG_aligned.npy', sound_suffix='Sound_aligned.npy', root_suffix='feature_normalized'):
    eeg_segments_list = []
    sound_segments_list = []
    subject_ids_list = []  # 新增一个列表来存储ID

    print(f"开始从根目录 '{root_dir}' 加载数据段及被试者ID...")
    subject_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

    for subject in tqdm(subject_folders, desc="处理被试者"):
        subject_path = os.path.join(root_dir, subject, root_suffix)
        eeg_files = glob.glob(os.path.join(subject_path, f'*{eeg_suffix}'))

        for eeg_file_path in eeg_files:
            base_name = os.path.basename(eeg_file_path).replace(eeg_suffix, '')
            sound_file_path = os.path.join(subject_path, base_name + sound_suffix)

            if os.path.exists(sound_file_path):
                try:
                    eeg_segment = np.load(eeg_file_path)
                    sound_segment = np.load(sound_file_path)

                    if eeg_segment.shape[0] == sound_segment.shape[0]:
                        eeg_segments_list.append(eeg_segment)
                        sound_segments_list.append(sound_segment)
                        subject_ids_list.append(subject)  # 关键：记录下当前数据段属于哪个被试者
                except Exception as e:
                    print(f"加载文件 {os.path.basename(eeg_file_path)} 时出错: {e}")

    print(f"\n加载完成！总共加载了 {len(eeg_segments_list)} 个数据段。")
    return eeg_segments_list, sound_segments_list, subject_ids_list

In [3]:
def zscore_time(X):
    """沿时间维(T, D)做zscore，避免0方差造成NaN。"""
    X = np.asarray(X, dtype=np.float64)
    mu = np.nanmean(X, axis=0, keepdims=True)
    sd = np.nanstd(X, axis=0, keepdims=True)
    sd[sd == 0] = 1.0
    return (X - mu) / sd

def fisher_z(r):
    r = np.clip(r, -0.999999, 0.999999)
    return 0.5 * np.log((1 + r) / (1 - r))

def inv_fisher_z(z):
    return (np.exp(2*z) - 1) / (np.exp(2*z) + 1)

def spearman_brown_correction(r_half):
    """Spearman-Brown 对半长估计的修正，得到全长可靠性。"""
    return (2 * r_half) / (1 + r_half) if np.isfinite(r_half) else np.nan

def split_half_reliability_time_series(X):
    """
    对 (T, D) 的时间序列做奇偶帧 split-half：
      1) 全段zscore（避免“分半各自标准化”抬高皮尔逊r）
      2) 奇帧 vs 偶帧，逐维计算 Pearson r
      3) 对逐维r做 Fisher-z 平均，再逆变换为 r_half
      4) Spearman-Brown 校正为 r_full
    返回: r_full (标量)
    """
    X = zscore_time(X)
    T = X.shape[0]
    if T < 4:
        return np.nan  # 太短
    X_odd = X[0::2, :]
    X_even = X[1::2, :]
    # 对齐长度
    m = min(X_odd.shape[0], X_even.shape[0])
    X_odd, X_even = X_odd[:m], X_even[:m]
    # 逐维相关
    rs = []
    for d in range(X.shape[1]):
        x = X_odd[:, d]
        y = X_even[:, d]
        if np.std(x) == 0 or np.std(y) == 0:
            continue
        r = np.corrcoef(x, y)[0, 1]
        if np.isfinite(r):
            rs.append(r)
    if len(rs) == 0:
        return np.nan
    r_half = inv_fisher_z(np.mean(fisher_z(np.array(rs))))
    r_full = spearman_brown_correction(r_half)
    return float(r_full)

def first_pc_time_series(X):
    """
    取 (T, D) 的第一主成分时间序列（PC1分数），先对时间做zscore。
    返回: (T,) 一维时间序列
    """
    Xz = zscore_time(X)
    # SVD: Xz = U S V^T; PC1的时间分数可取 U[:,0] * S[0]
    U, S, VT = np.linalg.svd(Xz, full_matrices=False)
    ts = U[:, 0] * S[0]
    return np.asarray(ts, dtype=np.float64)

def normxcorr_1d(x, y, max_lag):
    """
    归一化互相关（Pearson相关随时延），lag>0 表示 EEG 滞后于声音（声音领先）。
    返回: lags(np.arange(-max_lag, max_lag+1)), corrs(np.ndarray)
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    x = (x - x.mean()) / (x.std() if x.std() > 0 else 1.0)
    y = (y - y.mean()) / (y.std() if y.std() > 0 else 1.0)
    T = min(len(x), len(y))
    x, y = x[:T], y[:T]
    lags = np.arange(-max_lag, max_lag + 1, dtype=int)
    corrs = []
    for lag in lags:
        if lag >= 0:
            xx = x[lag:]
            yy = y[:T - lag]
        else:
            xx = x[:T + lag]
            yy = y[-lag:]
        if len(xx) < 4 or np.std(xx) == 0 or np.std(yy) == 0:
            corrs.append(np.nan)
        else:
            corrs.append(np.corrcoef(xx, yy)[0, 1])
    return lags, np.array(corrs, dtype=np.float64)

def circular_shift(arr, shift):
    """循环移位，正shift向后滚动。"""
    return np.roll(arr, shift)

# -----------------------------
# 1) 噪声上限：EEG/声音 split-half + 合成上限
# -----------------------------
def compute_noise_ceiling(eeg_segments, sound_segments, subject_ids):
    per_seg = []
    for i, (E, S, sid) in enumerate(tqdm(zip(eeg_segments, sound_segments, subject_ids),
                                        total=len(subject_ids), desc="Noise ceiling per segment")):
        r_eeg = split_half_reliability_time_series(E)
        r_snd = split_half_reliability_time_series(S)
        r_max = np.sqrt(r_eeg * r_snd) if (np.isfinite(r_eeg) and np.isfinite(r_snd) and r_eeg>0 and r_snd>0) else np.nan
        per_seg.append({"idx": i, "subject": sid, "r_eeg": r_eeg, "r_sound": r_snd, "r_max": r_max})

    # 整体（Fisher-z 加权平均）
    def fmean(key):
        vals = [p[key] for p in per_seg if np.isfinite(p[key])]
        if len(vals) == 0:
            return np.nan
        return float(inv_fisher_z(np.mean(fisher_z(np.clip(np.array(vals), -0.999999, 0.999999)))))

    overall = {
        "r_eeg_overall": fmean("r_eeg"),
        "r_sound_overall": fmean("r_sound")
    }
    # 上限的整体估计用几何平均：sqrt(mean_r_eeg * mean_r_sound)
    if np.isfinite(overall["r_eeg_overall"]) and np.isfinite(overall["r_sound_overall"]) \
       and overall["r_eeg_overall"] > 0 and overall["r_sound_overall"] > 0:
        overall["r_max_overall"] = float(np.sqrt(overall["r_eeg_overall"] * overall["r_sound_overall"]))
    else:
        overall["r_max_overall"] = np.nan

    # 按被试聚合（Fisher-z 后平均）
    per_subject = {}
    for key in ["r_eeg", "r_sound", "r_max"]:
        pass
    for rec in per_seg:
        sid = rec["subject"]
        per_subject.setdefault(sid, {"r_eeg": [], "r_sound": [], "r_max": []})
        for k in ["r_eeg", "r_sound", "r_max"]:
            if np.isfinite(rec[k]):
                per_subject[sid][k].append(rec[k])
    # 转为 Fisher-z 平均后的标量
    per_subject_stats = []
    for sid, d in per_subject.items():
        row = {"subject": sid}
        for k in ["r_eeg", "r_sound", "r_max"]:
            vals = d[k]
            if len(vals):
                row[k] = float(inv_fisher_z(np.mean(fisher_z(np.clip(np.array(vals), -0.999999, 0.999999)))))
            else:
                row[k] = np.nan
        per_subject_stats.append(row)

    print("\n=== 噪声上限（总体估计）===")
    print(json.dumps(overall, ensure_ascii=False, indent=2))

    return per_seg, per_subject_stats, overall

# -----------------------------
# 2) 时延对齐：PC1 互相关峰值 & 时间漂移置换
# -----------------------------
def compute_time_lag_alignment(eeg_segments, sound_segments, subject_ids,
                               frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33):
    max_lag_frames = max(1, int(round(max_lag_ms / frame_ms)))
    per_seg = []
    for i, (E, S, sid) in enumerate(tqdm(zip(eeg_segments, sound_segments, subject_ids),
                                        total=len(subject_ids), desc="Lag/cross-corr per segment")):
        try:
            eeg_ts = first_pc_time_series(E)
            snd_ts = first_pc_time_series(S)
            # 真实对齐的互相关
            lags, corrs = normxcorr_1d(eeg_ts, snd_ts, max_lag_frames)
            if np.all(~np.isfinite(corrs)):
                peak_r, peak_lag = np.nan, np.nan
            else:
                peak_idx = int(np.nanargmax(corrs))
                peak_r = float(corrs[peak_idx])
                peak_lag = int(lags[peak_idx])  # 单位：帧（>0 表示EEG滞后于声音）
            # 时间漂移（circular shift）无信息对照
            shift = max(1, int(round(len(snd_ts) * null_shift_ratio)))
            snd_shift = circular_shift(snd_ts, shift)
            _, corrs_null = normxcorr_1d(eeg_ts, snd_shift, max_lag_frames)
            peak_r_null = float(np.nanmax(corrs_null)) if np.any(np.isfinite(corrs_null)) else np.nan

            per_seg.append({
                "idx": i,
                "subject": sid,
                "peak_r": peak_r,
                "peak_lag_frames": peak_lag,
                "peak_lag_ms": (peak_lag * frame_ms) if np.isfinite(peak_lag) else np.nan,
                "peak_r_null": peak_r_null
            })
        except Exception as e:
            per_seg.append({"idx": i, "subject": sid, "peak_r": np.nan, "peak_lag_frames": np.nan,
                            "peak_lag_ms": np.nan, "peak_r_null": np.nan})

    # 汇总
    def median_key(key):
        vals = [p[key] for p in per_seg if np.isfinite(p[key])]
        return float(np.median(vals)) if len(vals) else np.nan

    summary = {
        "median_peak_r": median_key("peak_r"),
        "median_peak_lag_ms": median_key("peak_lag_ms"),
        "median_peak_r_null": median_key("peak_r_null"),
        "frac_r_gt_null": float(np.mean([
            (p["peak_r"] > p["peak_r_null"]) for p in per_seg
            if np.isfinite(p["peak_r"]) and np.isfinite(p["peak_r_null"])
        ])) if any(np.isfinite(p["peak_r"]) and np.isfinite(p["peak_r_null"]) for p in per_seg) else np.nan
    }

    print("\n=== 时延与对照（总体中位数）===")
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    print("注：peak_lag_ms > 0 表示 EEG 相对声音**滞后**（神经处理时延）；"
          "peak_lag_ms < 0 表示 EEG 领先声音（通常不符合生理预期）。")

    return per_seg, summary

In [4]:
root_dir = r'/content/drive/MyDrive/data'

In [5]:
eeg_segments, sound_segments, subject_ids = load_segments_with_subject_ids(root_dir)

# 1) 噪声上限
nc_per_seg, nc_per_subject, nc_overall = compute_noise_ceiling(eeg_segments, sound_segments, subject_ids)

# 2) 时延对齐（frame_ms=11 代表每帧约11ms；max_lag_ms=300 可按需调整）
lag_per_seg, lag_summary = compute_time_lag_alignment(
    eeg_segments, sound_segments, subject_ids,
    frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33
)


开始从根目录 '/content/drive/MyDrive/data' 加载数据段及被试者ID...


处理被试者: 100%|██████████| 20/20 [01:13<00:00,  3.70s/it]



加载完成！总共加载了 1794 个数据段。


Noise ceiling per segment:   0%|          | 0/1794 [00:00<?, ?it/s]/tmp/ipython-input-3058095305.py:4: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(X, axis=0, keepdims=True)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
Noise ceiling per segment: 100%|██████████| 1794/1794 [00:30<00:00, 57.97it/s]



=== 噪声上限（总体估计）===
{
  "r_eeg_overall": 0.9283464428682654,
  "r_sound_overall": 0.9330589277166446,
  "r_max_overall": 0.9306997026604366
}


Lag/cross-corr per segment: 100%|██████████| 1794/1794 [00:04<00:00, 367.07it/s]


=== 时延与对照（总体中位数）===
{
  "median_peak_r": NaN,
  "median_peak_lag_ms": NaN,
  "median_peak_r_null": NaN,
  "frac_r_gt_null": NaN
}
注：peak_lag_ms > 0 表示 EEG 相对声音**滞后**（神经处理时延）；peak_lag_ms < 0 表示 EEG 领先声音（通常不符合生理预期）。


In [6]:
# --- 安装（Colab通常自带这些）---
# !pip install -q numpy scipy tqdm

import os, glob, json
import numpy as np
from tqdm import tqdm

# ---------- 你之前的加载函数（保持一致） ----------
def load_segments_with_subject_ids(root_dir, eeg_suffix='EEG_aligned.npy', sound_suffix='Sound_aligned.npy', root_suffix='feature_normalized'):
    eeg_segments_list = []
    sound_segments_list = []
    subject_ids_list = []  # 新增一个列表来存储ID

    print(f"开始从根目录 '{root_dir}' 加载数据段及被试者ID...")
    subject_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

    for subject in tqdm(subject_folders, desc="处理被试者"):
        subject_path = os.path.join(root_dir, subject, root_suffix)
        eeg_files = glob.glob(os.path.join(subject_path, f'*{eeg_suffix}'))

        for eeg_file_path in eeg_files:
            base_name = os.path.basename(eeg_file_path).replace(eeg_suffix, '')
            sound_file_path = os.path.join(subject_path, base_name + sound_suffix)

            if os.path.exists(sound_file_path):
                try:
                    eeg_segment = np.load(eeg_file_path)
                    sound_segment = np.load(sound_file_path)

                    if eeg_segment.shape[0] == sound_segment.shape[0]:
                        eeg_segments_list.append(eeg_segment)
                        sound_segments_list.append(sound_segment)
                        subject_ids_list.append(subject)  # 关键：记录下当前数据段属于哪个被试者
                except Exception as e:
                    print(f"加载文件 {os.path.basename(eeg_file_path)} 时出错: {e}")

    print(f"\n加载完成！总共加载了 {len(eeg_segments_list)} 个数据段。")
    return eeg_segments_list, sound_segments_list, subject_ids_list

# ---------- 工具函数：zscore / 插值 / 可靠性 ----------
def zscore_time(X):
    X = np.asarray(X, dtype=np.float64)
    if X.size == 0:
        return X
    mu = np.nanmean(X, axis=0, keepdims=True)
    sd = np.nanstd(X, axis=0, keepdims=True)
    sd[(sd == 0) | ~np.isfinite(sd)] = 1.0
    Z = (X - mu) / sd
    Z[~np.isfinite(Z)] = 0.0
    return Z

def interp_nan_1d(x):
    x = np.asarray(x, dtype=np.float64)
    n = len(x)
    if n == 0:
        return x
    idx = np.arange(n)
    m = np.isfinite(x)
    if m.sum() == 0:
        return np.zeros_like(x)
    x2 = x.copy()
    # 线性插值内部 NaN；两端用最近有效值外推
    x2[~m] = np.interp(idx[~m], idx[m], x[m])
    return x2

def impute_timewise(X):
    X = np.asarray(X, dtype=np.float64)
    if X.size == 0:
        return X
    for d in range(X.shape[1]):
        col = X[:, d]
        if not np.all(np.isfinite(col)):
            col = interp_nan_1d(col)
        col[~np.isfinite(col)] = 0.0
        X[:, d] = col
    return X

def fisher_z(r):
    r = np.clip(r, -0.999999, 0.999999)
    return 0.5 * np.log((1 + r) / (1 - r))

def inv_fisher_z(z):
    return (np.exp(2*z) - 1) / (np.exp(2*z) + 1)

def spearman_brown(r_half):
    return (2 * r_half) / (1 + r_half) if np.isfinite(r_half) else np.nan

def split_half_odd_even(X):
    """奇偶帧 split-half（可能偏乐观）"""
    X = zscore_time(X)
    T, D = X.shape
    if T < 4 or D == 0:
        return np.nan
    Xo, Xe = X[0::2, :], X[1::2, :]
    m = min(len(Xo), len(Xe))
    if m < 2:
        return np.nan
    rs = []
    for d in range(D):
        x, y = Xo[:m, d], Xe[:m, d]
        if np.std(x) == 0 or np.std(y) == 0:
            continue
        r = np.corrcoef(x, y)[0, 1]
        if np.isfinite(r):
            rs.append(r)
    if not rs:
        return np.nan
    r_half = inv_fisher_z(np.mean(fisher_z(np.array(rs))))
    return spearman_brown(r_half)

def split_half_blocked(X, block_len=20):
    """分块 split-half：先按连续时间块求均值，再奇偶块相关（更保守）"""
    X = zscore_time(X)
    T, D = X.shape
    if T < block_len*2 or D == 0:
        return np.nan
    n_blocks = T // block_len
    Xb = X[:n_blocks*block_len].reshape(n_blocks, block_len, D).mean(axis=1)
    Xo, Xe = Xb[0::2, :], Xb[1::2, :]
    m = min(len(Xo), len(Xe))
    if m < 2:
        return np.nan
    rs = []
    for d in range(D):
        x, y = Xo[:m, d], Xe[:m, d]
        if np.std(x) == 0 or np.std(y) == 0:
            continue
        r = np.corrcoef(x, y)[0, 1]
        if np.isfinite(r):
            rs.append(r)
    if not rs:
        return np.nan
    r_half = inv_fisher_z(np.mean(fisher_z(np.array(rs))))
    return spearman_brown(r_half)

def first_pc_time_series_imputed(X):
    """对 (T,D) 做时间向插值 -> zscore -> SVD 取PC1（避免NaN导致SVD失败）。"""
    X = impute_timewise(X)
    X = zscore_time(X)
    if X.size == 0 or min(X.shape) < 2:
        return np.array([])
    try:
        U, S, VT = np.linalg.svd(X, full_matrices=False)
        return (U[:, 0] * S[0]).astype(np.float64)
    except Exception:
        return np.array([])

def normxcorr_1d(x, y, max_lag):
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    T = min(len(x), len(y))
    x, y = x[:T], y[:T]
    if T < max(4, max_lag+2):
        return np.array([]), np.array([])
    x = (x - x.mean()) / (x.std() if x.std() > 0 else 1.0)
    y = (y - y.mean()) / (y.std() if y.std() > 0 else 1.0)
    lags = np.arange(-max_lag, max_lag + 1, dtype=int)
    corrs = []
    for lag in lags:
        if lag >= 0:
            xx, yy = x[lag:], y[:T - lag]
        else:
            xx, yy = x[:T + lag], y[-lag:]
        if len(xx) < 4 or np.std(xx) == 0 or np.std(yy) == 0:
            corrs.append(np.nan)
        else:
            corrs.append(np.corrcoef(xx, yy)[0, 1])
    return lags, np.array(corrs, dtype=np.float64)

def circular_shift(arr, shift):
    return np.roll(arr, shift)

# ---------- 过滤与统计 ----------
def filter_and_summarize(eegs, snds, sids, min_frames=20):
    kept_e, kept_s, kept_id = [], [], []
    n_emptyT, n_emptyD, n_short = 0, 0, 0
    for E, S, sid in zip(eegs, snds, sids):
        if E.ndim != 2 or S.ndim != 2:
            n_emptyD += 1; continue
        T1, D1 = E.shape
        T2, D2 = S.shape
        if T1 == 0 or T2 == 0:
            n_emptyT += 1; continue
        if D1 == 0 or D2 == 0:
            n_emptyD += 1; continue
        if T1 < min_frames or T2 < min_frames:
            n_short += 1; continue
        kept_e.append(E); kept_s.append(S); kept_id.append(sid)
    print(f"过滤结果：总 {len(eegs)} 段 -> 保留 {len(kept_e)} 段 | 空T: {n_emptyT}, 空D: {n_emptyD}, 过短(<{min_frames}帧): {n_short}")
    return kept_e, kept_s, kept_id

def nan_inf_report(eegs, snds):
    def stats(arrs, name):
        n = len(arrs)
        any_nan = sum([np.isnan(a).any() for a in arrs])
        any_inf = sum([np.isinf(a).any() for a in arrs])
        print(f"{name}: 段数={n}, 含NaN段={any_nan} ({any_nan/n:.1%}), 含Inf段={any_inf} ({any_inf/n:.1%})")
    stats(eegs, "EEG"); stats(snds, "Sound")

# ---------- 主流程：噪声上限 & 时延 ----------
def compute_noise_ceiling(eeg_segments, sound_segments, subject_ids, method="both"):
    per_seg = []
    for i, (E, S, sid) in enumerate(tqdm(list(zip(eeg_segments, sound_segments, subject_ids)), desc="Noise ceiling per segment")):
        r_eeg_odd = split_half_odd_even(E)
        r_snd_odd = split_half_odd_even(S)
        r_eeg_blk = split_half_blocked(E, block_len=20)
        r_snd_blk = split_half_blocked(S, block_len=20)
        rec = {
            "idx": i, "subject": sid,
            "r_eeg_odd": r_eeg_odd, "r_sound_odd": r_snd_odd,
            "r_eeg_blk": r_eeg_blk, "r_sound_blk": r_snd_blk
        }
        per_seg.append(rec)

    def fisher_mean(vals):
        vals = [v for v in vals if np.isfinite(v)]
        return float(inv_fisher_z(np.mean(fisher_z(np.clip(np.array(vals), -0.999999, 0.999999))))) if vals else np.nan

    r_eeg_odd_all = fisher_mean([p["r_eeg_odd"] for p in per_seg])
    r_snd_odd_all = fisher_mean([p["r_sound_odd"] for p in per_seg])
    r_eeg_blk_all = fisher_mean([p["r_eeg_blk"] for p in per_seg])
    r_snd_blk_all = fisher_mean([p["r_sound_blk"] for p in per_seg])

    summary = {
        "r_eeg_odd_overall": r_eeg_odd_all,
        "r_sound_odd_overall": r_snd_odd_all,
        "r_max_odd_overall": np.sqrt(r_eeg_odd_all*r_snd_odd_all) if (np.isfinite(r_eeg_odd_all) and np.isfinite(r_snd_odd_all) and r_eeg_odd_all>0 and r_snd_odd_all>0) else np.nan,
        "r_eeg_blk_overall": r_eeg_blk_all,
        "r_sound_blk_overall": r_snd_blk_all,
        "r_max_blk_overall": np.sqrt(r_eeg_blk_all*r_snd_blk_all) if (np.isfinite(r_eeg_blk_all) and np.isfinite(r_snd_blk_all) and r_eeg_blk_all>0 and r_snd_blk_all>0) else np.nan
    }

    print("\n=== 噪声上限（总体估计）===")
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    return per_seg, summary

def compute_time_lag_alignment(eeg_segments, sound_segments, subject_ids,
                               frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33):
    max_lag_frames = max(1, int(round(max_lag_ms / frame_ms)))
    per_seg = []
    for i, (E, S, sid) in enumerate(tqdm(list(zip(eeg_segments, sound_segments, subject_ids)), desc="Lag/cross-corr per segment")):
        eeg_ts = first_pc_time_series_imputed(E)
        snd_ts = first_pc_time_series_imputed(S)
        lags, corrs = normxcorr_1d(eeg_ts, snd_ts, max_lag_frames)
        if lags.size == 0:
            per_seg.append({"idx": i, "subject": sid, "peak_r": np.nan, "peak_lag_ms": np.nan, "peak_r_null": np.nan})
            continue
        if np.all(~np.isfinite(corrs)):
            peak_r, peak_lag = np.nan, np.nan
        else:
            peak_idx = int(np.nanargmax(corrs))
            peak_r = float(corrs[peak_idx])
            peak_lag = int(lags[peak_idx])
        # 置换对照：循环移位
        shift = max(1, int(round(len(snd_ts) * null_shift_ratio)))
        snd_shift = circular_shift(snd_ts, shift)
        _, corrs_null = normxcorr_1d(eeg_ts, snd_shift, max_lag_frames)
        peak_r_null = float(np.nanmax(corrs_null)) if corrs_null.size and np.any(np.isfinite(corrs_null)) else np.nan

        per_seg.append({
            "idx": i, "subject": sid,
            "peak_r": peak_r,
            "peak_lag_ms": (peak_lag * frame_ms) if np.isfinite(peak_lag) else np.nan,
            "peak_r_null": peak_r_null
        })

    def median_key(key):
        vals = [p[key] for p in per_seg if np.isfinite(p[key])]
        return float(np.median(vals)) if vals else np.nan

    summary = {
        "median_peak_r": median_key("peak_r"),
        "median_peak_lag_ms": median_key("peak_lag_ms"),
        "median_peak_r_null": median_key("peak_r_null"),
        "frac_r_gt_null": float(np.mean([
            (p["peak_r"] > p["peak_r_null"]) for p in per_seg
            if np.isfinite(p["peak_r"]) and np.isfinite(p["peak_r_null"])
        ])) if any(np.isfinite(p["peak_r"]) and np.isfinite(p["peak_r_null"]) for p in per_seg) else np.nan
    }

    print("\n=== 时延与对照（总体中位数）===")
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    print("注：peak_lag_ms > 0 表示 EEG 相对声音**滞后**；peak_lag_ms < 0 表示 EEG 领先声音。")
    return per_seg, summary

# ---------- 运行 ----------
  # <- 你的数据根目录

# 加载
eeg_segments, sound_segments, subject_ids = load_segments_with_subject_ids(root_dir)

# 先过滤：去掉空段/短段（<20帧≈220ms，可按需要调大到 50 帧）
eeg_segments, sound_segments, subject_ids = filter_and_summarize(eeg_segments, sound_segments, subject_ids, min_frames=20)

# 段级 NaN/Inf 报告（便于确认问题规模）
nan_inf_report(eeg_segments, sound_segments)

# 1) 噪声上限（提供奇偶与分块两种）
nc_per_seg, nc_summary = compute_noise_ceiling(eeg_segments, sound_segments, subject_ids)

# 2) 时延对齐（PC1 插值后再做互相关；默认 ±300ms）
lag_per_seg, lag_summary = compute_time_lag_alignment(
    eeg_segments, sound_segments, subject_ids,
    frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33
)

print("\n完成。若仍出现大量 NaN，请把 nan_inf_report 的比例和一个具体段的形状/NaN分布发我看。")


开始从根目录 '/content/drive/MyDrive/data' 加载数据段及被试者ID...


处理被试者: 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]



加载完成！总共加载了 1794 个数据段。
过滤结果：总 1794 段 -> 保留 1794 段 | 空T: 0, 空D: 0, 过短(<20帧): 0
EEG: 段数=1794, 含NaN段=0 (0.0%), 含Inf段=0 (0.0%)
Sound: 段数=1794, 含NaN段=1794 (100.0%), 含Inf段=0 (0.0%)


Noise ceiling per segment:   0%|          | 0/1794 [00:00<?, ?it/s]/tmp/ipython-input-4006312740.py:45: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(X, axis=0, keepdims=True)
Noise ceiling per segment: 100%|██████████| 1794/1794 [00:49<00:00, 35.93it/s]



=== 噪声上限（总体估计）===
{
  "r_eeg_odd_overall": 0.9283464428682654,
  "r_sound_odd_overall": 0.9330681531757942,
  "r_max_odd_overall": 0.930704303715423,
  "r_eeg_blk_overall": 0.7387319816317919,
  "r_sound_blk_overall": -0.08746792723618271,
  "r_max_blk_overall": NaN
}


Lag/cross-corr per segment: 100%|██████████| 1794/1794 [01:11<00:00, 25.22it/s]


=== 时延与对照（总体中位数）===
{
  "median_peak_r": 0.1367003934743225,
  "median_peak_lag_ms": 55.0,
  "median_peak_r_null": 0.14892065585877712,
  "frac_r_gt_null": 0.4793756967670011
}
注：peak_lag_ms > 0 表示 EEG 相对声音**滞后**；peak_lag_ms < 0 表示 EEG 领先声音。

完成。若仍出现大量 NaN，请把 nan_inf_report 的比例和一个具体段的形状/NaN分布发我看。


In [7]:
import numpy as np, json
from tqdm import tqdm

# ---------- 1) NaN 诊断 ----------
def nan_breakdown_per_segment(S):
    """返回每段的：整体NaN率、每列NaN率向量。"""
    S = np.asarray(S)
    nan_mask = ~np.isfinite(S)
    overall = nan_mask.mean() if S.size else 1.0
    per_col = nan_mask.mean(axis=0) if S.ndim==2 and S.shape[1]>0 else np.array([])
    return overall, per_col

def summarize_sound_nan(sound_segments):
    overall_rates = []
    percol_accum_sum = None
    percol_accum_cnt = None
    D_max = max(s.shape[1] for s in sound_segments if s.ndim==2) if sound_segments else 0

    for S in sound_segments:
        overall, per_col = nan_breakdown_per_segment(S)
        overall_rates.append(overall)
        # 用最长维度对齐，缺的列用NaN填（不计入分母）
        if per_col.size:
            if percol_accum_sum is None:
                percol_accum_sum = np.zeros(D_max, dtype=float)
                percol_accum_cnt = np.zeros(D_max, dtype=float)
            d = per_col.size
            percol_accum_sum[:d] += per_col
            percol_accum_cnt[:d] += 1.0

    print("\n=== 声音特征 NaN 概览 ===")
    print(f"- 段级：含NaN的段 = {sum(r>0 for r in overall_rates)}/{len(overall_rates)} "
          f"({sum(r>0 for r in overall_rates)/len(overall_rates):.1%}); "
          f"中位数NaN率 = {np.median(overall_rates):.2%}")
    if percol_accum_cnt is not None:
        percol_mean = percol_accum_sum / np.maximum(percol_accum_cnt, 1e-9)
        top_bad = np.argsort(-percol_mean)[:10]
        print(f"- 列级（跨段平均）前10个最差列及其NaN率：")
        for j in top_bad:
            print(f"  列{j}: 平均NaN率={percol_mean[j]:.2%}")
        return percol_mean
    else:
        return None

# ---------- 2) 丢弃坏列 + 时间插值修复 ----------
def interp_nan_1d(x):
    x = np.asarray(x, dtype=np.float64)
    n = len(x)
    if n == 0: return x
    idx = np.arange(n)
    m = np.isfinite(x)
    if m.sum() == 0:
        return np.zeros_like(x)
    y = x.copy()
    y[~m] = np.interp(idx[~m], idx[m], x[m])  # 线性插值，端点外推
    return y

def repair_sound_matrix(S, drop_thr=0.6):
    """
    对单段声音特征 (T,D)：丢弃NaN占比 > drop_thr 的列；其余列做时间向插值，返回修复后的矩阵与保留列掩码。
    drop_thr=0.6 表示“这一列 >60% 的帧是NaN”则视为坏列（可按需调大/调小）。
    """
    S = np.asarray(S, dtype=np.float64)
    T, D = S.shape
    col_nan_rate = np.mean(~np.isfinite(S), axis=0)
    keep_mask = col_nan_rate <= drop_thr
    if keep_mask.sum() == 0:
        return np.zeros((T,0), dtype=np.float64), keep_mask
    S2 = S[:, keep_mask].copy()
    for d in range(S2.shape[1]):
        S2[:, d] = interp_nan_1d(S2[:, d])
    # z-score over time（避免尺度差）
    mu = S2.mean(axis=0, keepdims=True)
    sd = S2.std(axis=0, keepdims=True); sd[sd==0] = 1.0
    S2 = (S2 - mu) / sd
    return S2, keep_mask

def batch_repair_sound(sound_segments, drop_thr=0.6):
    repaired = []
    dropped_cols_stat = []
    for S in tqdm(sound_segments, desc="修复声音特征"):
        S2, keep_mask = repair_sound_matrix(S, drop_thr=drop_thr)
        repaired.append(S2)
        dropped_cols_stat.append({"D_in": S.shape[1], "D_kept": int(keep_mask.sum())})
    print("修复完成：平均保留列数 =",
          np.mean([r["D_kept"] for r in dropped_cols_stat]),
          " / 平均原始列数 =",
          np.mean([r["D_in"] for r in dropped_cols_stat]))
    return repaired, dropped_cols_stat

# ---------- 3) PC1 & envelope 时间序列 ----------
def first_pc_time_series(X):
    X = np.asarray(X, dtype=np.float64)
    if X.size == 0 or min(X.shape) < 2:
        return np.array([])
    # SVD取PC1
    U,S,VT = np.linalg.svd(X, full_matrices=False)
    return (U[:,0]*S[0]).astype(np.float64)

def envelope_from_matrix(S):
    """
    通用 envelope 代理：行向量的L2范数（或能量），再z-score。
    对没有明确“能量/0阶MFCC”的特征组也适用。
    """
    if S.size == 0: return np.array([])
    e = np.sqrt((S**2).sum(axis=1))
    e = (e - e.mean()) / (e.std() if e.std()>0 else 1.0)
    return e

# ---------- 4) 更稳健的噪声上限（针对1D时间序列） ----------
def split_half_1d(ts, block_len=None):
    ts = np.asarray(ts, dtype=np.float64)
    if ts.size < 8: return np.nan
    # z-score
    ts = (ts - np.nanmean(ts)) / (np.nanstd(ts) if np.nanstd(ts)>0 else 1.0)
    if block_len is None:
        # 奇偶帧
        x, y = ts[0::2], ts[1::2]
    else:
        n_blocks = len(ts)//block_len
        if n_blocks < 4: return np.nan
        B = ts[:n_blocks*block_len].reshape(n_blocks, block_len).mean(axis=1)
        x, y = B[0::2], B[1::2]
    if len(x) < 3 or len(y) < 3: return np.nan
    r_half = np.corrcoef(x, y)[0,1]
    if not np.isfinite(r_half): return np.nan
    # Spearman-Brown
    return (2*r_half)/(1+r_half)

def fisher_mean(rs):
    rs = [r for r in rs if np.isfinite(r)]
    if not rs: return np.nan
    z = np.arctanh(np.clip(rs, -0.999999, 0.999999))
    return float(np.tanh(np.mean(z)))

def noise_ceiling_from_pc1(eeg_segments, sound_repaired):
    eeg_r_blk, snd_r_blk = [], []
    for E, S2 in zip(eeg_segments, sound_repaired):
        eeg_pc1 = first_pc_time_series(E)
        snd_pc1 = first_pc_time_series(S2)
        eeg_r_blk.append(split_half_1d(eeg_pc1, block_len=20))
        snd_r_blk.append(split_half_1d(snd_pc1, block_len=20))
    r_eeg = fisher_mean(eeg_r_blk)
    r_snd = fisher_mean(snd_r_blk)
    r_max = np.sqrt(r_eeg*r_snd) if (np.isfinite(r_eeg) and np.isfinite(r_snd) and r_eeg>0 and r_snd>0) else np.nan
    print("\n=== 基于 PC1 的分块噪声上限 ===")
    print(json.dumps({"r_eeg_blk_overall_PC1": r_eeg, "r_sound_blk_overall_PC1": r_snd, "r_max_blk_overall_PC1": r_max},
                     ensure_ascii=False, indent=2))
    return r_eeg, r_snd, r_max

# ---------- 5) 时延互相关（PC1 & envelope） ----------
def normxcorr_1d(x, y, max_lag):
    x = np.asarray(x, dtype=np.float64); y = np.asarray(y, dtype=np.float64)
    T = min(len(x), len(y))
    if T < max(4, max_lag+2): return np.array([]), np.array([])
    x = (x - x.mean()) / (x.std() if x.std()>0 else 1.0)
    y = (y - y.mean()) / (y.std() if y.std()>0 else 1.0)
    lags = np.arange(-max_lag, max_lag+1)
    corrs = []
    for lag in lags:
        if lag >= 0:
            xx, yy = x[lag:], y[:T-lag]
        else:
            xx, yy = x[:T+lag], y[-lag:]
        if len(xx) < 4 or xx.std()==0 or yy.std()==0:
            corrs.append(np.nan)
        else:
            corrs.append(np.corrcoef(xx, yy)[0,1])
    return lags, np.array(corrs)

def lag_summary_from_series(eeg_ts, snd_ts, frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33):
    max_lag = int(round(max_lag_ms/frame_ms))
    lags, corrs = normxcorr_1d(eeg_ts, snd_ts, max_lag)
    if lags.size==0 or np.all(~np.isfinite(corrs)):
        return {"peak_r": np.nan, "peak_lag_ms": np.nan, "peak_r_null": np.nan}
    peak_idx = int(np.nanargmax(corrs))
    peak_r = float(corrs[peak_idx]); peak_lag_ms = float(lags[peak_idx]*frame_ms)
    # null
    shift = max(1, int(round(len(snd_ts)*null_shift_ratio)))
    snd_shift = np.roll(snd_ts, shift)
    _, corrs_null = normxcorr_1d(eeg_ts, snd_shift, max_lag)
    peak_r_null = float(np.nanmax(corrs_null)) if np.any(np.isfinite(corrs_null)) else np.nan
    return {"peak_r": peak_r, "peak_lag_ms": peak_lag_ms, "peak_r_null": peak_r_null}

def compute_lag_stats(eeg_segments, sound_repaired, frame_ms=11.0, max_lag_ms=300):
    rows_pc1, rows_env = [], []
    for E, S2 in tqdm(list(zip(eeg_segments, sound_repaired)), desc="Lag PC1 / Envelope"):
        eeg_pc1 = first_pc_time_series(E)
        snd_pc1 = first_pc_time_series(S2)
        env_ts = envelope_from_matrix(S2)
        rows_pc1.append(lag_summary_from_series(eeg_pc1, snd_pc1, frame_ms, max_lag_ms))
        rows_env.append(lag_summary_from_series(eeg_pc1, env_ts, frame_ms, max_lag_ms))
    def summarize(rows):
        med = lambda k: float(np.median([r[k] for r in rows if np.isfinite(r[k])])) if any(np.isfinite(r[k]) for r in rows) else np.nan
        frac = np.mean([r["peak_r"]>r["peak_r_null"] for r in rows if np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"])]) \
               if any(np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"]) for r in rows) else np.nan
        return {"median_peak_r": med("peak_r"), "median_peak_lag_ms": med("peak_lag_ms"),
                "median_peak_r_null": med("peak_r_null"), "frac_r_gt_null": float(frac) if np.isfinite(frac) else np.nan}
    print("\n=== 时延（PC1 对 PC1）===")
    print(json.dumps(summarize(rows_pc1), ensure_ascii=False, indent=2))
    print("\n=== 时延（PC1 对 Envelope）===")
    print(json.dumps(summarize(rows_env), ensure_ascii=False, indent=2))
    return rows_pc1, rows_env

# ====== 执行：先看NaN分布，再修复，再重算 ======
percol_nan_mean = summarize_sound_nan(sound_segments)

sound_repaired, drop_stats = batch_repair_sound(sound_segments, drop_thr=0.6)

# 基于 PC1 的分块噪声上限（更接近真实上限）
_ = noise_ceiling_from_pc1(eeg_segments, sound_repaired)

# 重算时延（PC1↔PC1 与 PC1↔envelope 两套）
_ = compute_lag_stats(eeg_segments, sound_repaired, frame_ms=11.0, max_lag_ms=300)



=== 声音特征 NaN 概览 ===
- 段级：含NaN的段 = 1794/1794 (100.0%); 中位数NaN率 = 16.67%
- 列级（跨段平均）前10个最差列及其NaN率：
  列52: 平均NaN率=99.69%
  列51: 平均NaN率=99.22%
  列50: 平均NaN率=99.22%
  列48: 平均NaN率=99.22%
  列45: 平均NaN率=99.22%
  列46: 平均NaN率=99.22%
  列47: 平均NaN率=99.22%
  列49: 平均NaN率=99.22%
  列53: 平均NaN率=99.22%
  列0: 平均NaN率=0.00%


修复声音特征: 100%|██████████| 1794/1794 [00:02<00:00, 733.91it/s]


修复完成：平均保留列数 = 45.062430323299886  / 平均原始列数 = 54.0

=== 基于 PC1 的分块噪声上限 ===
{
  "r_eeg_blk_overall_PC1": 0.7056337694877208,
  "r_sound_blk_overall_PC1": 0.27392708407007016,
  "r_max_blk_overall_PC1": 0.4396500891585755
}


Lag PC1 / Envelope: 100%|██████████| 1794/1794 [01:49<00:00, 16.42it/s]


=== 时延（PC1 对 PC1）===
{
  "median_peak_r": 0.13992195481928765,
  "median_peak_lag_ms": 55.0,
  "median_peak_r_null": 0.1432376058130258,
  "frac_r_gt_null": 0.4938684503901895
}

=== 时延（PC1 对 Envelope）===
{
  "median_peak_r": 0.12740620356650428,
  "median_peak_lag_ms": 44.0,
  "median_peak_r_null": 0.1391428877698161,
  "frac_r_gt_null": 0.48940914158305465
}


In [8]:
import numpy as np, json
from tqdm import tqdm

# --------- 工具：EEG高通 + 包络 ----------
def highpass_moving_average(ts, win=15):
    ts = np.asarray(ts, dtype=np.float64)
    if ts.size == 0: return ts
    if ts.size < 2*win:
        ts = (ts - ts.mean()) / (ts.std() if ts.std()>0 else 1.0)
        return ts
    kernel = np.ones(win, dtype=np.float64)/win
    trend = np.convolve(ts, kernel, mode='same')
    hp = ts - trend
    hp = (hp - hp.mean()) / (hp.std() if hp.std()>0 else 1.0)
    return hp

def first_pc_time_series(X):
    X = np.asarray(X, dtype=np.float64)
    if X.size == 0 or min(X.shape) < 2: return np.array([])
    U,S,VT = np.linalg.svd(X, full_matrices=False)
    return (U[:,0]*S[0]).astype(np.float64)

def envelope_from_matrix(S):
    if S.size == 0: return np.array([])
    e = np.sqrt((S**2).sum(axis=1))
    e = (e - e.mean()) / (e.std() if e.std()>0 else 1.0)
    return e

# --------- 用“高NaN列”自动推断 voicing 列，并为每段生成有声帧掩码 ----------
def compute_percol_nan(sound_segments):
    D = max(S.shape[1] for S in sound_segments if S.ndim==2) if sound_segments else 0
    sums = np.zeros(D); cnts = np.zeros(D)
    for S in sound_segments:
        if S.ndim!=2: continue
        d = S.shape[1]
        sums[:d] += (~np.isfinite(S)).mean(axis=0)
        cnts[:d] += 1
    mean_nan = sums / np.maximum(cnts, 1e-9)
    return mean_nan

try:
    percol_nan_mean  # 如果你之前已计算过，就用已有的
except NameError:
    percol_nan_mean = compute_percol_nan(sound_segments)

# 经验阈值：把“跨段平均 NaN 率 > 0.95”的列当成 F0/颤动/抖动等voicing相关列
voicing_cols = np.where(percol_nan_mean > 0.95)[0]
print(f"推断出的 voicing 相关列索引（用于掩码）：{voicing_cols.tolist()}")

def voiced_mask_from_original(S, voicing_cols, fallback_percentile=40):
    T, D = S.shape
    if len(voicing_cols):
        vc = voicing_cols[voicing_cols < D]  # 防越界
        if len(vc):
            # 任一 voicing 列为有限值 => 认为该帧“有声”
            m = np.any(np.isfinite(S[:, vc]), axis=1)
            # 若全False，用fallback
            if m.any():
                return m.astype(bool)
    # fallback：用能量代理（来自修复后矩阵或原矩阵的L2）
    e = np.sqrt((np.nan_to_num(S)**2).sum(axis=1))
    thr = np.percentile(e[~np.isnan(e)], fallback_percentile) if np.any(~np.isnan(e)) else 0.0
    return (e > thr)

# --------- 有声帧加权的互相关（只在y=声音上有声帧处计算） ----------
def masked_normxcorr_1d(x, y, mask_y, max_lag, min_eff=10):
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    mask_y = np.asarray(mask_y, dtype=bool)
    T = min(len(x), len(y), len(mask_y))
    if T < max(4, max_lag+2): return np.array([]), np.array([])
    x = (x - x.mean()) / (x.std() if x.std()>0 else 1.0)
    y = (y - y.mean()) / (y.std() if y.std()>0 else 1.0)
    lags = np.arange(-max_lag, max_lag+1, dtype=int)
    corrs = []
    for lag in lags:
        if lag >= 0:
            xx, yy = x[lag:T], y[:T-lag]
            my = mask_y[:T-lag]
        else:
            xx, yy = x[:T+lag], y[-lag:T]
            my = mask_y[-lag:T]
        # 只取有声帧
        if len(yy)==0:
            corrs.append(np.nan); continue
        sel = my
        if sel.sum() < min_eff:
            corrs.append(np.nan); continue
        xs = xx[sel]; ys = yy[sel]
        if len(xs)<2 or xs.std()==0 or ys.std()==0:
            corrs.append(np.nan); continue
        corrs.append(np.corrcoef(xs, ys)[0,1])
    return lags, np.array(corrs, dtype=np.float64)

def lag_summary_masked(eeg_ts, snd_ts, mask_y, frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33):
    max_lag = int(round(max_lag_ms/frame_ms))
    lags, corrs = masked_normxcorr_1d(eeg_ts, snd_ts, mask_y, max_lag)
    if lags.size==0 or np.all(~np.isfinite(corrs)):
        return {"peak_r": np.nan, "peak_lag_ms": np.nan, "peak_r_null": np.nan}
    peak_idx = int(np.nanargmax(corrs))
    peak_r = float(corrs[peak_idx]); peak_lag_ms = float(lags[peak_idx]*frame_ms)
    # null（循环移位保持自相关）
    shift = max(1, int(round(len(snd_ts)*null_shift_ratio)))
    snd_shift = np.roll(snd_ts, shift)
    _, corrs_null = masked_normxcorr_1d(eeg_ts, snd_shift, mask_y, max_lag)
    peak_r_null = float(np.nanmax(corrs_null)) if np.any(np.isfinite(corrs_null)) else np.nan
    return {"peak_r": peak_r, "peak_lag_ms": peak_lag_ms, "peak_r_null": peak_r_null}

# --------- 运行：用 有声帧掩码 + 高通EEG 复算（PC1↔PC1 和 PC1↔envelope） ----------
def summarize(rows):
    med = lambda k: float(np.median([r[k] for r in rows if np.isfinite(r[k])])) if any(np.isfinite(r[k]) for r in rows) else np.nan
    frac = np.mean([r["peak_r"]>r["peak_r_null"] for r in rows if np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"])]) \
           if any(np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"]) for r in rows) else np.nan
    return {"median_peak_r": med("peak_r"), "median_peak_lag_ms": med("peak_lag_ms"),
            "median_peak_r_null": med("peak_r_null"), "frac_r_gt_null": float(frac) if np.isfinite(frac) else np.nan}

frame_ms = 11.0
max_lag_ms = 300

rows_pc1, rows_env = [], []
for E, S_orig, S_rep in tqdm(list(zip(eeg_segments, sound_segments, sound_repaired)), desc="Voicing-masked lag"):
    # 时间序列
    eeg_pc1 = first_pc_time_series(E)
    eeg_hp  = highpass_moving_average(eeg_pc1, win=15)  # ~165ms窗口
    snd_pc1 = first_pc_time_series(S_rep)
    snd_env = envelope_from_matrix(S_rep)
    # 掩码（基于原始声学矩阵）
    vm = voiced_mask_from_original(S_orig, voicing_cols, fallback_percentile=40)
    rows_pc1.append(lag_summary_masked(eeg_hp, snd_pc1, vm, frame_ms, max_lag_ms))
    rows_env.append(lag_summary_masked(eeg_hp, snd_env, vm, frame_ms, max_lag_ms))

print("\n=== 有声帧掩码 + EEG高通：时延（PC1 ↔ PC1）===")
print(json.dumps(summarize(rows_pc1), ensure_ascii=False, indent=2))
print("\n=== 有声帧掩码 + EEG高通：时延（PC1 ↔ Envelope）===")
print(json.dumps(summarize(rows_env), ensure_ascii=False, indent=2))


推断出的 voicing 相关列索引（用于掩码）：[45, 46, 47, 48, 49, 50, 51, 52, 53]


Voicing-masked lag: 100%|██████████| 1794/1794 [01:56<00:00, 15.39it/s]


=== 有声帧掩码 + EEG高通：时延（PC1 ↔ PC1）===
{
  "median_peak_r": 0.1648615358271223,
  "median_peak_lag_ms": 0.0,
  "median_peak_r_null": 0.15878776916996157,
  "frac_r_gt_null": 0.5468227424749164
}

=== 有声帧掩码 + EEG高通：时延（PC1 ↔ Envelope）===
{
  "median_peak_r": 0.1355609136848986,
  "median_peak_lag_ms": -11.0,
  "median_peak_r_null": 0.12734341613428096,
  "frac_r_gt_null": 0.5362318840579711
}


In [13]:
import numpy as np
from tqdm import tqdm

def tukey_window(n, alpha=0.25):
    if n <= 1: return np.ones(n)
    w = np.ones(n)
    edge = int(alpha*(n-1)/2)
    ramp = 0.5*(1 - np.cos(np.pi*np.arange(edge+1)/edge)) if edge>0 else np.array([])
    w[:edge+1] = ramp
    w[-(edge+1):] = ramp[::-1]
    return w

def prep_ts(ts, taper=True):
    ts = np.asarray(ts, dtype=np.float64)
    ts = (ts - ts.mean()) / (ts.std() if ts.std()>0 else 1.0)
    if taper:
        w = tukey_window(len(ts), alpha=0.25)
        ts = ts * w
    return ts

def normxcorr_1d(x, y, max_lag, min_eff=12):
    """
    归一化互相关（Pearson相关随时延），lag>0 表示 EEG 滞后于声音。
    加强版：严格检查 xx/yy 的长度与方差，避免 17 vs 0 等长度不匹配导致的报错。
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    T = min(len(x), len(y))
    x, y = x[:T], y[:T]

    # 如果整体有效帧太少，直接放弃
    if T < max(min_eff + 1, max_lag + 2):
        return np.array([]), np.array([])

    # 标准化
    sx, sy = x.std(), y.std()
    x = (x - x.mean()) / (sx if np.isfinite(sx) and sx > 0 else 1.0)
    y = (y - y.mean()) / (sy if np.isfinite(sy) and sy > 0 else 1.0)

    lags = np.arange(-max_lag, max_lag + 1, dtype=int)
    corrs = []
    for lag in lags:
        if lag >= 0:
            xx, yy = x[lag:], y[:T - lag]
        else:
            xx, yy = x[:T + lag], y[-lag:]

        # 关键：双向最小长度与方差检查
        if len(xx) < min_eff or len(yy) < min_eff:
            corrs.append(np.nan); continue
        sxx, syy = xx.std(), yy.std()
        if not np.isfinite(sxx) or not np.isfinite(syy) or sxx == 0 or syy == 0:
            corrs.append(np.nan); continue

        corrs.append(np.corrcoef(xx, yy)[0, 1])

    return lags, np.array(corrs, dtype=np.float64)


# 取你已有的序列生成方法（来自上一个单元）
eeg_pc1 = first_pc_time_series(E); eeg_hp = highpass_moving_average(eeg_pc1, 15)
snd_pc1 = first_pc_time_series(S_rep); snd_env = envelope_from_matrix(S_rep)
vm = voiced_mask_from_original(S_orig, voicing_cols, fallback_percentile=40)

def masked_series(x, y, mask_y):
    # 只保留有声帧；两端缺失自动剔除
    m = mask_y[:min(len(x), len(y))].astype(bool)
    return x[:len(m)][m], y[:len(m)][m]

frame_ms = 11.0
max_lag_ms = 500  # 扩到 ±500ms
max_lag = int(round(max_lag_ms/frame_ms))

rows = []
for E, S_orig, S_rep in tqdm(list(zip(eeg_segments, sound_segments, sound_repaired))[:], desc="Lag sanity (wide)"):
    eeg_pc1 = first_pc_time_series(E)
    eeg_hp  = highpass_moving_average(eeg_pc1, win=15)
    snd_env = envelope_from_matrix(S_rep)
    vm = voiced_mask_from_original(S_orig, voicing_cols, 40)

    x, y = masked_series(prep_ts(eeg_hp, True), prep_ts(snd_env, True), vm)

    # 我们的定义：lag>0 表示 EEG 落后于 声音
    lags1, corrs1 = normxcorr_1d(x, y, max_lag)
    lags2, corrs2 = normxcorr_1d(y, x, max_lag)  # 互换顺序（检查符号一致性）

    def peak(lags, cs):
        if cs.size==0 or np.all(~np.isfinite(cs)): return np.nan, np.nan
        i = int(np.nanargmax(cs)); return float(cs[i]), float(lags[i]*frame_ms)

    peak_r1, peak_lag1_ms = peak(lags1, corrs1)  # 期望正值 ~ +50~200
    peak_r2, peak_lag2_ms = peak(lags2, corrs2)  # 期望负值（符号相反）

    rows.append((peak_r1, peak_lag1_ms, peak_r2, peak_lag2_ms))

arr = np.array([[*r] for r in rows if np.all(np.isfinite(r))])
if arr.size:
    print("定义1（EEG滞后为正）:  中位r/中位滞后(ms) =", np.median(arr[:,0]), np.median(arr[:,1]))
    print("定义2（反向计算，应为相反符号）: 中位r/中位滞后(ms) =", np.median(arr[:,2]), np.median(arr[:,3]))
else:
    print("没有可汇总样本。")


Lag sanity (wide): 100%|██████████| 1794/1794 [01:26<00:00, 20.82it/s]

定义1（EEG滞后为正）:  中位r/中位滞后(ms) = 0.15257807144787666 33.0
定义2（反向计算，应为相反符号）: 中位r/中位滞后(ms) = 0.15257807144787666 -33.0


In [14]:
valid = [np.isfinite(p[0]) and np.isfinite(p[1]) and np.isfinite(p[2]) and np.isfinite(p[3]) for p in rows]
print(f"有效段数：{sum(valid)}/{len(rows)}")


有效段数：1689/1794


In [15]:
def channelwise_best_lag(E, S_orig, S_rep, frame_ms=11.0, max_lag_ms=300):
    max_lag = int(round(max_lag_ms/frame_ms))
    vm = voiced_mask_from_original(S_orig, voicing_cols, 40)
    y = envelope_from_matrix(S_rep)
    y = prep_ts(y, True)
    best = {"peak_r": np.nan, "lag_ms": np.nan, "ch": -1, "null": np.nan}
    for ch in range(E.shape[1]):
        x = highpass_moving_average((E[:, ch]-E[:, ch].mean())/(E[:, ch].std() or 1), win=15)
        x = prep_ts(x, True)
        # 掩码
        mlen = min(len(x), len(y), len(vm))
        x2, y2, vm2 = x[:mlen], y[:mlen], vm[:mlen]
        lags, cs = normxcorr_1d(x2[vm2], y2[vm2], max_lag)
        if cs.size==0 or np.all(~np.isfinite(cs)): continue
        i = int(np.nanargmax(cs)); r, lag = float(cs[i]), float(lags[i]*frame_ms)
        # null
        shift = max(1, int(round(len(y2)*0.33)))
        ysh = np.roll(y2, shift)
        _, csn = normxcorr_1d(x2[vm2], ysh[vm2], max_lag)
        rnull = float(np.nanmax(csn)) if np.any(np.isfinite(csn)) else np.nan
        if not np.isfinite(best["peak_r"]) or r > best["peak_r"]:
            best = {"peak_r": r, "lag_ms": lag, "ch": ch, "null": rnull}
    return best

best_rows = []
for E, S_orig, S_rep in tqdm(list(zip(eeg_segments, sound_segments, sound_repaired)), desc="Channelwise ROI"):
    best_rows.append(channelwise_best_lag(E, S_orig, S_rep, frame_ms=11.0, max_lag_ms=300))

vals = [r for r in best_rows if np.isfinite(r["peak_r"]) and np.isfinite(r["lag_ms"]) and np.isfinite(r["null"])]
if vals:
    med_r = float(np.median([v["peak_r"] for v in vals]))
    med_lag = float(np.median([v["lag_ms"] for v in vals]))
    frac = float(np.mean([v["peak_r"]>v["null"] for v in vals]))
    print({"median_peak_r": med_r, "median_peak_lag_ms": med_lag, "frac_r_gt_null": frac})
    # 可选：看“最好通道”的分布（是否集中某些通道）
    from collections import Counter
    cnt = Counter([v["ch"] for v in vals])
    print("Top channels:", cnt.most_common(10))
else:
    print("没有可用的通道结果。")


Channelwise ROI: 100%|██████████| 1794/1794 [17:01<00:00,  1.76it/s]

{'median_peak_r': 0.2111433337421394, 'median_peak_lag_ms': 22.0, 'frac_r_gt_null': 0.9111238532110092}
Top channels: [(10, 134), (1, 109), (8, 89), (5, 85), (17, 75), (4, 75), (21, 75), (13, 72), (28, 69), (14, 68)]


In [16]:
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

def channelwise_best_lag_with_sid(E, S_orig, S_rep, sid, frame_ms=11.0, max_lag_ms=300):
    max_lag = int(round(max_lag_ms/frame_ms))
    # envelope + voicing 面具
    env = envelope_from_matrix(S_rep)
    vm  = voiced_mask_from_original(S_orig, voicing_cols, 40)
    env = (env - env.mean()) / (env.std() if env.std()>0 else 1.0)
    best = {"subject": sid, "peak_r": np.nan, "lag_ms": np.nan, "ch": -1, "null": np.nan}
    for ch in range(E.shape[1]):
        x = (E[:,ch]-E[:,ch].mean())/(E[:,ch].std() if E[:,ch].std()>0 else 1.0)
        x = highpass_moving_average(x, 15); x = prep_ts(x, True)
        y = prep_ts(env, True)
        mlen = min(len(x), len(y), len(vm)); x2,y2,vm2 = x[:mlen], y[:mlen], vm[:mlen]
        lags, cs = normxcorr_1d(x2[vm2], y2[vm2], int(max_lag))
        if cs.size==0 or np.all(~np.isfinite(cs)): continue
        i = int(np.nanargmax(cs)); r, lag = float(cs[i]), float(lags[i]*frame_ms)
        # null
        ysh = np.roll(y2, max(1, int(0.33*len(y2))))
        _, csn = normxcorr_1d(x2[vm2], ysh[vm2], int(max_lag))
        rnull = float(np.nanmax(csn)) if np.any(np.isfinite(csn)) else np.nan
        if not np.isfinite(best["peak_r"]) or r > best["peak_r"]:
            best.update({"peak_r": r, "lag_ms": lag, "ch": ch, "null": rnull})
    return best

rows = []
for E,S0,Sr,sid in tqdm(list(zip(eeg_segments, sound_segments, sound_repaired, subject_ids)), desc="Per-subject ROI"):
    rows.append(channelwise_best_lag_with_sid(E,S0,Sr,sid))

# 被试级统计
per_subj = defaultdict(list)
for r in rows:
    if np.isfinite(r["peak_r"]) and np.isfinite(r["null"]):
        per_subj[r["subject"]].append(r)

summary = {}
for sid, lst in per_subj.items():
    med_r  = float(np.median([x["peak_r"] for x in lst]))
    med_l  = float(np.median([x["lag_ms"] for x in lst if np.isfinite(x["lag_ms"])])) if any(np.isfinite(x["lag_ms"]) for x in lst) else np.nan
    frac   = float(np.mean([x["peak_r"] > x["null"] for x in lst]))
    ch_cnt = Counter([x["ch"] for x in lst])
    summary[sid] = {"median_peak_r": med_r, "median_lag_ms": med_l, "frac_r_gt_null": frac, "top_channels": ch_cnt.most_common(5)}

# 总结打印
print("被试级中位数（前5个示例）：")
for sid in list(summary.keys())[:5]:
    print(sid, summary[sid])


Per-subject ROI: 100%|██████████| 1794/1794 [17:02<00:00,  1.75it/s]

被试级中位数（前5个示例）：
CLIP_VIE_RechLn4_Cer&HUS19m_EP004 {'median_peak_r': 0.20859937185613167, 'median_lag_ms': 44.0, 'frac_r_gt_null': 0.9145299145299145, 'top_channels': [(4, 12), (17, 10), (14, 9), (5, 9), (10, 9)]}
CLIP_VIE_RechLn4_Cer&DENGH23m_EP017 {'median_peak_r': 0.21620414013553896, 'median_lag_ms': 11.0, 'frac_r_gt_null': 0.9026548672566371, 'top_channels': [(30, 11), (1, 9), (7, 8), (23, 8), (20, 7)]}
CLIP_VIE_RechLn4_Cer&LIS22m_EP013 {'median_peak_r': 0.2050682365697435, 'median_lag_ms': 5.5, 'frac_r_gt_null': 0.9152542372881356, 'top_channels': [(21, 9), (13, 9), (5, 7), (30, 7), (24, 7)]}
CLIP_VIE_RechLn4_Cer&LIY25m_EP002 {'median_peak_r': 0.22677340070081276, 'median_lag_ms': 22.0, 'frac_r_gt_null': 0.9008264462809917, 'top_channels': [(28, 10), (8, 10), (17, 8), (5, 8), (21, 7)]}
CLIP_VIE_RechLn4_Cer&NIUZ19m_EP010 {'median_peak_r': 0.20391481612874848, 'median_lag_ms': 0.0, 'frac_r_gt_null': 0.9024390243902439, 'top_channels': [(2, 13), (4, 12), (8, 12), (27, 9), (10, 9)]}


In [17]:
def causal_ma(ts, win=9):
    # 单向（因果）均值滤波，避免零相位；win≈100ms 对11ms帧
    ts = np.asarray(ts, dtype=np.float64)
    if ts.size==0: return ts
    out = np.empty_like(ts)
    csum = 0.0
    for i in range(len(ts)):
        csum += ts[i]
        if i >= win: csum -= ts[i-win]
        out[i] = csum / min(i+1, win)
    out = (out - out.mean()) / (out.std() if out.std()>0 else 1.0)
    return out

rows_causal = []
for E,S0,Sr in tqdm(list(zip(eeg_segments, sound_segments, sound_repaired)), desc="Causal envelope lag"):
    env = envelope_from_matrix(Sr)
    env_causal = causal_ma(env, win=9)  # ≈99ms
    vm  = voiced_mask_from_original(S0, voicing_cols, 40)
    eeg_pc1 = first_pc_time_series(E)
    eeg_hp  = highpass_moving_average(eeg_pc1, 15)
    # 掩码后互相关
    res = lag_summary_masked(eeg_hp, env_causal, vm, frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33)
    rows_causal.append(res)

def _summ(rows):
    med = lambda k: float(np.median([r[k] for r in rows if np.isfinite(r[k])])) if any(np.isfinite(r[k]) for r in rows) else np.nan
    frac = float(np.mean([r["peak_r"]>r["peak_r_null"] for r in rows if np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"])])) \
           if any(np.isfinite(r["peak_r"]) and np.isfinite(r["peak_r_null"]) for r in rows) else np.nan
    return {"median_peak_r": med("peak_r"), "median_peak_lag_ms": med("peak_lag_ms"),
            "median_peak_r_null": med("peak_r_null"), "frac_r_gt_null": frac}

print("因果包络：", _summ(rows_causal))


Causal envelope lag: 100%|██████████| 1794/1794 [00:55<00:00, 32.43it/s]

因果包络： {'median_peak_r': 0.10885543520635788, 'median_peak_lag_ms': -22.0, 'median_peak_r_null': 0.10629090321731183, 'frac_r_gt_null': 0.5234113712374582}


In [18]:
import numpy as np
from tqdm import tqdm

def build_lagged_design(s, lags):
    T = len(s); X = np.zeros((T, len(lags)))
    for i,L in enumerate(lags):
        if L>=0: X[L:, i] = s[:T-L]
        else:    X[:T+L, i] = s[-L:]
    return X

def trf_ridge(X, y, alpha=10.0):
    XtX = X.T @ X; XtY = X.T @ y
    return np.linalg.solve(XtX + alpha*np.eye(X.shape[1]), XtY)

rng = np.random.default_rng(0)
idx = rng.choice(len(eeg_segments), size=min(300, len(eeg_segments)), replace=False)

lags_ms = np.arange(-200, 401, 11)   # -200…+400 ms
lags_fr = (lags_ms/11.0).astype(int)

res = []
for k in tqdm(idx, desc="Mini-TRF"):
    E,S0,Sr = eeg_segments[k], sound_segments[k], sound_repaired[k]
    env = envelope_from_matrix(Sr)
    env = causal_ma(env, 9)  # 用因果包络
    vm  = voiced_mask_from_original(S0, voicing_cols, 40)

    # 选该段的最佳通道（注意：正式建模时请在训练折上选 ROI，避免泄漏）
    best = channelwise_best_lag_with_sid(E,S0,Sr,subject_ids[k])
    ch = best["ch"] if best["ch"]>=0 else 0
    y = (E[:,ch]-E[:,ch].mean())/(E[:,ch].std() if E[:,ch].std()>0 else 1.0)

    X = build_lagged_design(env, lags_fr)
    mlen = min(len(y), X.shape[0], len(vm))
    y = y[:mlen]; X = X[:mlen][vm[:mlen]]
    if len(y)<200 or X.shape[0]<=X.shape[1]: continue
    y = y[:len(X)]

    n = len(y); ntr = int(0.8*n)
    w = trf_ridge(X[:ntr], y[:ntr], alpha=10.0)
    yhat = X[ntr:] @ w
    r = np.corrcoef(y[ntr:], yhat)[0,1] if y[ntr:].std()>0 and yhat.std()>0 else np.nan

    # 置换对照（循环移位保持自相关）
    yperm = np.roll(y, int(0.33*len(y)))
    w0 = trf_ridge(X[:ntr], yperm[:ntr], alpha=10.0)
    yhat0 = X[ntr:] @ w0
    r0 = np.corrcoef(yperm[ntr:], yhat0)[0,1] if yperm[ntr:].std()>0 and yhat0.std()>0 else np.nan

    # TRF峰（正滞后侧）
    pos = lags_ms >= 0
    lag_peak_ms = float(lags_ms[pos][np.argmax(np.abs(w[pos]))]) if np.any(pos) else np.nan
    res.append((r, r0, lag_peak_ms))

arr = np.array([x for x in res if np.all(np.isfinite(x))])
if arr.size:
    print({"median_pred_r": float(np.median(arr[:,0])),
           "median_pred_r_null": float(np.median(arr[:,1])),
           "frac_r_better_than_null": float(np.mean(arr[:,0]>arr[:,1])),
           "median_trf_peak_ms_(>=0)": float(np.median(arr[:,2]))})
else:
    print("TRF样本不足。")


Mini-TRF: 100%|██████████| 300/300 [02:58<00:00,  1.68it/s]

{'median_pred_r': 0.026339256870607897, 'median_pred_r_null': 0.027859566415895466, 'frac_r_better_than_null': 0.5379061371841155, 'median_trf_peak_ms_(>=0)': 295.0}


In [19]:
# ===========================================
# TRF (0..200ms) with ROI-in-train, z-score-in-train, Ridge + 2nd-diff smoothing
# ===========================================
import numpy as np, json, math
from tqdm import tqdm
from collections import defaultdict, Counter
from sklearn.model_selection import GroupKFold

# ---------- 基础工具 ----------
def zscore1d(x):
    x = np.asarray(x, dtype=np.float64)
    m, s = x.mean(), x.std()
    return (x - m) / (s if s>0 and np.isfinite(s) else 1.0), m, (s if s>0 and np.isfinite(s) else 1.0)

def highpass_moving_average(ts, win=15):
    ts = np.asarray(ts, dtype=np.float64)
    if ts.size == 0: return ts
    if ts.size < 2*win:
        return (ts - ts.mean())/(ts.std() if ts.std()>0 else 1.0)
    kernel = np.ones(win, dtype=np.float64)/win
    trend = np.convolve(ts, kernel, mode='same')
    hp = ts - trend
    return (hp - hp.mean())/(hp.std() if hp.std()>0 else 1.0)

def envelope_from_matrix(S):
    if S.size == 0: return np.array([])
    e = np.sqrt((S**2).sum(axis=1))
    return (e - e.mean())/(e.std() if e.std()>0 else 1.0)

def causal_ma(ts, win=9):
    ts = np.asarray(ts, dtype=np.float64)
    if ts.size==0: return ts
    out = np.empty_like(ts)
    csum = 0.0
    for i in range(len(ts)):
        csum += ts[i]
        if i >= win: csum -= ts[i-win]
        out[i] = csum / min(i+1, win)
    return (out - out.mean())/(out.std() if out.std()>0 else 1.0)

def normxcorr_1d(x, y, max_lag, min_eff=12):
    x = np.asarray(x, dtype=np.float64); y = np.asarray(y, dtype=np.float64)
    T = min(len(x), len(y)); x, y = x[:T], y[:T]
    if T < max(min_eff + 1, max_lag + 2): return np.array([]), np.array([])
    x = (x - x.mean())/(x.std() if x.std()>0 else 1.0)
    y = (y - y.mean())/(y.std() if y.std()>0 else 1.0)
    lags = np.arange(-max_lag, max_lag+1, dtype=int)
    cs = []
    for lag in lags:
        if lag>=0: xx, yy = x[lag:], y[:T-lag]
        else:      xx, yy = x[:T+lag], y[-lag:]
        if len(xx) < min_eff or len(yy) < min_eff: cs.append(np.nan); continue
        sx, sy = xx.std(), yy.std()
        if sx==0 or sy==0 or not np.isfinite(sx) or not np.isfinite(sy): cs.append(np.nan); continue
        cs.append(np.corrcoef(xx, yy)[0,1])
    return lags, np.array(cs, dtype=np.float64)

# 你之前推断的 voicing 列 -> 用它从原始 Sound 估计“有声帧”掩码
def voiced_mask_from_original(S_orig, voicing_cols, fallback_percentile=40):
    T, D = S_orig.shape
    if len(voicing_cols):
        vc = [c for c in voicing_cols if c < D]
        if len(vc):
            m = np.any(np.isfinite(S_orig[:, vc]), axis=1)
            if m.any():
                return m.astype(bool)
    e = np.sqrt((np.nan_to_num(S_orig)**2).sum(axis=1))
    thr = np.percentile(e[~np.isnan(e)], fallback_percentile) if np.any(~np.isnan(e)) else 0.0
    return (e > thr)

# ---------- 设计矩阵：滞后 envelope + 一列 voicing ----------
def build_lagged_design(env, lags_fr, voicing=None):
    T = len(env)
    X = np.zeros((T, len(lags_fr)), dtype=np.float64)
    for i,L in enumerate(lags_fr):
        if L>=0:
            X[L:, i] = env[:T-L]
        else:
            X[:T+L, i] = env[-L:]
    if voicing is None:
        return X
    v = np.asarray(voicing, dtype=np.float64).reshape(-1,1)
    v = v[:T]
    return np.hstack([X[:T], v])

# ---------- 二阶差分平滑矩阵 (只作用于滞后列) ----------
def diff2_mat(L):
    if L < 3:
        return np.zeros((L, L), dtype=np.float64)
    D = np.zeros((L-2, L), dtype=np.float64)
    for i in range(L-2):
        D[i, i]   = 1.0
        D[i, i+1] = -2.0
        D[i, i+2] = 1.0
    return D.T @ D  # (L,L)

# ---------- 带平滑的岭回归（滞后列 + voicing 列） ----------
def trf_fit_ridge_smooth(X_tr, y_tr, alpha=10.0, beta=10.0, n_lag_cols=None, alpha_voicing=None):
    """
    X_tr: (N, L + 1)  L列为滞后；最后1列为 voicing
    y_tr: (N,)
    正则： (X^T X + alpha*I + beta*D^T D) w = X^T y
    其中 D^T D 只作用于前 L 列；voicing 列只用 alpha（或 alpha_voicing）
    """
    N, P = X_tr.shape
    L = int(n_lag_cols if n_lag_cols is not None else P-1)
    assert L <= P
    XtX = X_tr.T @ X_tr
    XtY = X_tr.T @ y_tr
    # 基础岭
    A = XtX + alpha * np.eye(P)
    # 平滑只加到滞后子块
    if beta > 0 and L >= 3:
        D2 = diff2_mat(L)
        A[:L, :L] += beta * D2
    # voicing 列单独alpha（可与 alpha 相同）
    if alpha_voicing is not None and P > L:
        A[L, L] += (alpha_voicing - alpha)
    w = np.linalg.solve(A, XtY)
    return w  # (P,)

# ---------- 列标准化（只用训练集拟合） ----------
def fit_standardizer(X):
    mu = X.mean(axis=0)
    sd = X.std(axis=0)
    sd[(~np.isfinite(sd)) | (sd==0)] = 1.0
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# ---------- 训练集内选 ROI（Top-k 通道） ----------
def select_roi_channels_for_subject(train_idx, top_k=3, frame_ms=11.0, max_lag_ms=200):
    max_lag = int(round(max_lag_ms/frame_ms))
    # 按通道累计“段内最佳相关”的中位数
    # 首先确定通道数
    D = eeg_segments[train_idx[0]].shape[1]
    per_ch_rs = [[] for _ in range(D)]
    for i in train_idx:
        E = eeg_segments[i]; S0 = sound_segments[i]; Sr = sound_repaired[i]
        vm = voiced_mask_from_original(S0, voicing_cols, 40)
        env = causal_ma(envelope_from_matrix(Sr), win=9)  # 因果包络
        yenv = (env - env.mean())/(env.std() if env.std()>0 else 1.0)
        for ch in range(D):
            y = (E[:,ch]-E[:,ch].mean())/(E[:,ch].std() if E[:,ch].std()>0 else 1.0)
            y = highpass_moving_average(y, 15)
            # 只在有声帧上算互相关，找峰
            mlen = min(len(y), len(yenv), len(vm))
            yy, ee, mm = y[:mlen], yenv[:mlen], vm[:mlen]
            lags, cs = normxcorr_1d(yy[mm], ee[mm], max_lag)
            if cs.size and np.any(np.isfinite(cs)):
                per_ch_rs[ch].append(float(np.nanmax(cs)))
    # 通道得分 = 中位数相关
    ch_scores = np.array([np.median(rs) if len(rs) else -np.inf for rs in per_ch_rs])
    roi = np.argsort(-ch_scores)[:top_k]
    return roi.tolist()

# ---------- 构建一个被试的 K 折评估（嵌套/选择 α, β） ----------
def eval_subject_trf(subject_id, frame_ms=11.0):
    # 索引
    idx = [i for i, sid in enumerate(subject_ids) if sid == subject_id]
    n_segs = len(idx)
    if n_segs < 8:
        return {"subject": subject_id, "note": "too few segments"}
    groups = np.arange(n_segs)  # 每段一个组
    outer = GroupKFold(n_splits=min(5, n_segs))
    # 滞后窗
    lags_ms = np.arange(0, 201, 11)  # 0..200ms
    lags_fr = (lags_ms / frame_ms).astype(int)
    L = len(lags_fr)
    alpha_grid = [1.0, 10.0, 100.0]
    beta_grid  = [0.0, 10.0, 100.0]

    seg_corrs = []      # 每段的预测相关
    seg_corrs_null = [] # 置换对照
    peak_lags = []      # 每折（每ROI/模型）的峰滞后（取绝对最大权重的滞后）

    for tr_idx_rel, va_idx_rel in outer.split(np.zeros(n_segs), groups=groups):
        tr_idx = [idx[i] for i in tr_idx_rel]
        va_idx = [idx[i] for i in va_idx_rel]

        # —— 在训练折内选 ROI（Top-3 通道）——
        roi = select_roi_channels_for_subject(tr_idx, top_k=3, frame_ms=frame_ms, max_lag_ms=200)

        # —— 构建训练/验证设计矩阵（拼接所有段）——
        def build_dataset(seg_indices):
            X_list, y_list = [], []
            for k in seg_indices:
                E = eeg_segments[k]; S0 = sound_segments[k]; Sr = sound_repaired[k]
                env = causal_ma(envelope_from_matrix(Sr), win=9)
                vm  = voiced_mask_from_original(S0, voicing_cols, 40)  # 只用于附加列，不做删样本
                Xseg = build_lagged_design(env, lags_fr, voicing=vm)
                # 目标：ROI 平均（或先z后平均）
                ychs = []
                for ch in roi:
                    y = (E[:,ch]-E[:,ch].mean())/(E[:,ch].std() if E[:,ch].std()>0 else 1.0)
                    y = highpass_moving_average(y, 15)
                    ychs.append(y[:Xseg.shape[0]])
                yseg = np.mean(np.vstack(ychs), axis=0) if len(ychs) else np.zeros(Xseg.shape[0])
                # 截齐长度
                T = min(Xseg.shape[0], len(yseg))
                X_list.append(Xseg[:T]); y_list.append(yseg[:T])
            X = np.vstack(X_list); y = np.concatenate(y_list)
            return X, y

        Xtr_raw, ytr_raw = build_dataset(tr_idx)
        Xva_raw_list, yva_raw_list, seg_len_list = [], [], []
        for k in va_idx:
            Xk, yk = build_dataset([k])
            Xva_raw_list.append(Xk); yva_raw_list.append(yk); seg_len_list.append(len(yk))

        # —— 训练集列标准化、y 去均值（按训练折拟合）——
        Xmu, Xsd = fit_standardizer(Xtr_raw)
        Xtr = apply_standardizer(Xtr_raw, Xmu, Xsd)
        ytr, ymu, ysd = zscore1d(ytr_raw)  # 仅训练折去均值并缩放

        # 内部切分（80/20）做一个简易的嵌套 CV 选 α,β
        n = len(ytr)
        ntr = int(0.8*n)
        Xtr_i, ytr_i = Xtr[:ntr], ytr[:ntr]
        Xvl_i, yvl_i = Xtr[ntr:], ytr[ntr:]
        best_score, best_hp = -np.inf, (10.0, 10.0)
        for alpha in alpha_grid:
            for beta in beta_grid:
                w = trf_fit_ridge_smooth(Xtr_i, ytr_i, alpha=alpha, beta=beta, n_lag_cols=L, alpha_voicing=alpha)
                yhat = Xvl_i @ w
                r = np.corrcoef(yvl_i, yhat)[0,1] if yvl_i.std()>0 and yhat.std()>0 else -np.inf
                if np.isfinite(r) and r > best_score:
                    best_score, best_hp = r, (alpha, beta)
        alpha, beta = best_hp

        # —— 用最佳超参在“整个训练折”上重训 ——
        w = trf_fit_ridge_smooth(Xtr, ytr, alpha=alpha, beta=beta, n_lag_cols=L, alpha_voicing=alpha)

        # 峰滞后（只看滞后列）
        lag_block = w[:L]
        pk = int(np.argmax(np.abs(lag_block))) if L else 0
        peak_lags.append(float((pk) * 11.0))  # 每列步长≈11ms

        # —— 在验证段上评估（逐段相关 + 置换）——
        for Xk_raw, yk_raw in zip(Xva_raw_list, yva_raw_list):
            Xk = apply_standardizer(Xk_raw, Xmu, Xsd)
            # 验证 y 用训练均值/方差做同样变换（仅去训练均值，缩放到训练尺度）
            yk = (yk_raw - ymu) / (ysd if ysd>0 else 1.0)
            yhat = Xk @ w
            rk = np.corrcoef(yk, yhat)[0,1] if yk.std()>0 and yhat.std()>0 else np.nan

            # 置换：循环移位 y
            shift = max(1, int(0.33*len(yk)))
            yperm = np.roll(yk, shift)
            yhat0 = Xk @ w
            rk0 = np.corrcoef(yperm, yhat0)[0,1] if yperm.std()>0 and yhat0.std()>0 else np.nan

            seg_corrs.append(rk); seg_corrs_null.append(rk0)

    # 汇总
    seg_corrs = np.array(seg_corrs, dtype=np.float64)
    seg_corrs_null = np.array(seg_corrs_null, dtype=np.float64)
    valid = np.isfinite(seg_corrs) & np.isfinite(seg_corrs_null)
    res = {
        "subject": subject_id,
        "n_segments_eval": int(valid.sum()),
        "median_pred_r": float(np.median(seg_corrs[valid])) if valid.any() else np.nan,
        "median_pred_r_null": float(np.median(seg_corrs_null[valid])) if valid.any() else np.nan,
        "frac_r_better_than_null": float(np.mean(seg_corrs[valid] > seg_corrs_null[valid])) if valid.any() else np.nan,
        "median_trf_peak_ms_(>=0)": float(np.median(peak_lags)) if len(peak_lags) else np.nan,
        "note": f"ROI top-3 | lags 0..200ms | alpha,beta grid={alpha_grid}x{beta_grid}"
    }
    return res

# ---------- 对所有被试运行 ----------
subjects = sorted(set(subject_ids))
all_res = []
for sid in tqdm(subjects, desc="TRF per subject"):
    all_res.append(eval_subject_trf(sid, frame_ms=11.0))

# 打印摘要
print("\n=== TRF（被试级）摘要（前5条）===")
for r in all_res[:5]:
    print(r["subject"], {k:r[k] for k in r if k!="subject"})

# 全体汇总
vals = [r for r in all_res if np.isfinite(r["median_pred_r"]) and np.isfinite(r["median_pred_r_null"])]
if vals:
    med_r  = float(np.median([r["median_pred_r"] for r in vals]))
    med_r0 = float(np.median([r["median_pred_r_null"] for r in vals]))
    frac   = float(np.mean([r["median_pred_r"] > r["median_pred_r_null"] for r in vals]))
    med_pk = float(np.median([r["median_trf_peak_ms_(>=0)"] for r in vals if np.isfinite(r["median_trf_peak_ms_(>=0)"])])) if any(np.isfinite(r["median_trf_peak_ms_(>=0)"]) for r in vals) else np.nan
    print("\n=== TRF（总体）===")
    print(json.dumps({
        "median_of_subject_medians": med_r,
        "median_of_subject_nulls": med_r0,
        "frac_subjects_better_than_null": frac,
        "median_TRF_peak_ms_(>=0)": med_pk
    }, ensure_ascii=False, indent=2))
else:
    print("没有有效的被试结果。")


TRF per subject: 100%|██████████| 15/15 [23:48<00:00, 95.22s/it]


=== TRF（被试级）摘要（前5条）===
CLIP_VIE_RechLn4_Cer&DENGH23m_EP017 {'n_segments_eval': 115, 'median_pred_r': 0.036103696770945735, 'median_pred_r_null': 0.0032201849572996172, 'frac_r_better_than_null': 0.5478260869565217, 'median_trf_peak_ms_(>=0)': 143.0, 'note': 'ROI top-3 | lags 0..200ms | alpha,beta grid=[1.0, 10.0, 100.0]x[0.0, 10.0, 100.0]'}
CLIP_VIE_RechLn4_Cer&HUS19m_EP004 {'n_segments_eval': 120, 'median_pred_r': 0.007931682675624552, 'median_pred_r_null': 0.0017481332753566154, 'frac_r_better_than_null': 0.55, 'median_trf_peak_ms_(>=0)': 198.0, 'note': 'ROI top-3 | lags 0..200ms | alpha,beta grid=[1.0, 10.0, 100.0]x[0.0, 10.0, 100.0]'}
CLIP_VIE_RechLn4_Cer&LIS22m_EP013 {'n_segments_eval': 122, 'median_pred_r': 0.011886997247697475, 'median_pred_r_null': -0.005712400374946643, 'frac_r_better_than_null': 0.5737704918032787, 'median_trf_peak_ms_(>=0)': 143.0, 'note': 'ROI top-3 | lags 0..200ms | alpha,beta grid=[1.0, 10.0, 100.0]x[0.0, 10.0, 100.0]'}
CLIP_VIE_RechLn4_Cer&LIY25m_EP002 

In [ ]:
# Pipeline function
def analyze_eeg_sound_data(root_dir, frame_ms=11.0, max_lag_ms=300, null_shift_ratio=0.33, min_frames=20, drop_thr=0.6):
    """
    分析 EEG 和 Sound 数据，计算噪声上限、时延对齐、通道级 ROI 和 TRF。

    Args:
        root_dir (str): 数据根目录。
        frame_ms (float): 每帧的毫秒数。
        max_lag_ms (int): 互相关计算的最大时延（毫秒）。
        null_shift_ratio (float): 置换对照的循环移位比例。
        min_frames (int): 过滤短于此帧数的数据段。
        drop_thr (float): 声音特征列的 NaN 丢弃阈值。

    Returns:
        dict: 包含各项分析结果的字典。
    """
    print("--- 开始数据分析管线 ---")

    # 1. 加载数据
    eeg_segments, sound_segments, subject_ids = load_segments_with_subject_ids(root_dir)

    # 2. 过滤空段/短段
    eeg_segments, sound_segments, subject_ids = filter_and_summarize(
        eeg_segments, sound_segments, subject_ids, min_frames=min_frames
    )

    # 报告 NaN/Inf
    nan_inf_report(eeg_segments, sound_segments)

    # 3. 修复声音特征 (丢弃坏列 + 时间插值)
    percol_nan_mean = summarize_sound_nan(sound_segments)
    sound_repaired, drop_stats = batch_repair_sound(sound_segments, drop_thr=drop_thr)

    # 4. 计算基于 PC1 的噪声上限
    _, _, nc_pc1_overall = noise_ceiling_from_pc1(eeg_segments, sound_repaired)

    # 5. 计算时延对齐 (PC1 vs PC1, PC1 vs Envelope)
    lag_pc1_per_seg, lag_pc1_summary = compute_lag_stats(
        eeg_segments, sound_repaired, frame_ms=frame_ms, max_lag_ms=max_lag_ms
    )

    # 6. 计算通道级 ROI (Envelope)
    best_channels_summary = compute_lag_stats_channelwise(
         eeg_segments, sound_segments, sound_repaired, frame_ms=frame_ms, max_lag_ms=max_lag_ms, null_shift_ratio=null_shift_ratio, voicing_cols=np.where(percol_nan_mean > 0.95)[0] # 使用推断出的 voicing 列
    )

    # 7. 计算 TRF (被试级 K 折)
    trf_results_per_subject = run_trf_analysis_per_subject(
        eeg_segments, sound_segments, sound_repaired, subject_ids,
        voicing_cols=np.where(percol_nan_mean > 0.95)[0], # 使用推断出的 voicing 列
        frame_ms=frame_ms, max_lag_ms_roi=200, lags_ms_trf=np.arange(0, 201, 11)
    )

    print("\n--- 数据分析管线完成 ---")

    return {
        "noise_ceiling_pc1": nc_pc1_overall,
        "lag_alignment_pc1": lag_pc1_summary,
        "channel_roi_summary": best_channels_summary,
        "trf_results_per_subject": trf_results_per_subject,
        "sound_nan_summary": {"per_segment_rates": [summarize_sound_nan(s)[0] for s in sound_segments], "per_col_mean": percol_nan_mean.tolist()},
        "sound_repair_stats": drop_stats
    }

# 运行管线
# root_dir = r'/content/drive/MyDrive/data' # 请确保 root_dir 已定义
# analysis_results = analyze_eeg_sound_data(root_dir)

# 打印总体结果摘要 (如果需要)
# print("\n=== 总体结果摘要 ===")
# print(json.dumps({
#     "Noise Ceiling (PC1)": analysis_results["noise_ceiling_pc1"],
#     "Lag Alignment (PC1)": analysis_results["lag_alignment_pc1"],
#     "Channel ROI Summary": analysis_results["channel_roi_summary"]["overall_summary"], # 假设返回字典中有 overall_summary
#     "TRF Overall": analysis_results["trf_results_per_subject"]["overall_summary"] # 假设返回字典中有 overall_summary
# }, ensure_ascii=False, indent=2))